In [1]:
## Configs
FROM_DIR="../resouces/data/CSI300news_chunked_summarized"
TO_DIR="../resouces/data/CSI300news_chunked_summarized_senti"
BERT_BACKBONE_DIR = "../resources/open_models/FinBert"
CLASSIFIER_DIR = '../resources/ckpts/FinBert'

In [ ]:
## 初始化
import sys; sys.path.append("..")
from utils.models import *
from transformers import AutoTokenizer, AutoModel
import time
import pandas as pd
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained(BERT_BACKBONE_DIR)
backbone = AutoModel.from_pretrained(BERT_BACKBONE_DIR)
model = BaseModel.use_subclass("bert_classifier")(backbone,2)
model.load_classifier(CLASSIFIER_DIR)

/mnt/disk2/xinghua.jia/.miniforge3/envs/finbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classifier weights loaded from ../resources/ckpts/FinBert/classifier_weights.pth


In [5]:
def fn_extract(news):
    input_tokens = tokenizer(news, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return model.pred(input_tokens)

def extract_sentiment_from_news(from_dir, to_dir, fn_extract):
    os.makedirs(to_dir,exist_ok=True)
    done_files = os.listdir(to_dir)
    total_time = 0  # 总耗时
    total_count = 0  # 总条数
    start_time = time.time()  # 开始时间
    for filename in os.listdir(from_dir):
        stock_code = filename.split('.')[0]
        if not filename.endswith(".json"): continue
        # if stock_id+".json" in done_files: continue

        print(f"当前股票代码：{stock_code}")
        filepath = os.path.join(from_dir, filename)
        # 读取文件
        df = pd.read_json(filepath)
        df['date'] = df['date'].dt.strftime("%Y-%m-%d %H:%M:%S")
        # 处理每行数据
        for index, row in tqdm(df.iterrows()):
            df.loc[index, 'sentiment'] = fn_extract(row['chunk'])
            total_count += 1

        print(f"{stock_code} done.")  # 打印或保存结果DataFrame
        df.to_json(os.path.join(to_dir,f'{stock_code}.json'), force_ascii=False, orient='records', indent=2)
        
    end_time = time.time()  # 结束时间
    total_time = end_time - start_time  # 总耗时（秒）

    print(f"总耗时：{total_time:.2f} 秒")
    print(f"总条数：{total_count} 条")



In [9]:
fn_extract(["盛运环保2月13日晚间发布公告称，截至目前，共有37.48亿元到期债务未清偿。", "真好啊"])

(array([[0.8732003 , 0.1267997 ],
        [0.00231368, 0.99768627]], dtype=float32),
 ['NEGATIVE', 'POSITIVE'])